This notebook has been developed for a code-along introductory seminar on using [brightway](https://docs.brightway.dev/en/latest/). You can a more complete guide on brightway [here](https://learn.brightway.dev/en/latest/content/notebooks/BW25_for_beginners.html).

More information on the seminars can be found [here](https://adelinejerome.github.io/seminars-for-esa/).

# Importing packages

Importing brightway:

In [1]:
import bw2data as bd

Additionally, we import two other packages:

- `pandas` for manipulating tables,
- `matplotlib` for generating figures.

# Set up your project

The structure of information is similar to the Activity Browser. The information is structured in different projects, which contains several databases. To get the list of existing projects, use:

To see which project is currenly loaded:

To load another project or to create a new project if the project name is not already in the list of exiting projects:

In [2]:
bd.projects.set_current("brightway_demo")

To delete a project, use `bd.projects.delete_project`but it returns an error if the project does not exist. With the function below, it tries to delete a project but does nothing if an error is returned.

In [6]:
def reset_brightway_project(project_name):
    try:
        bd.projects.delete_project(project_name, True)
    except ValueError:
        pass

# Manage and import databases

To see the list of databases in the current project, use:

## Import the "biosphere" database

In [ ]:
bi.bw2setup()

Save the biosphere database as `biosphere_db`.

## Import ecoinvent

When the biosphere database is imported, the ecoinvent database can be imported as well. The importer uses the folder "datasets" from the extracted files of the .7z file which can be fount on the [ecoinvent website](https://ecoquery.ecoinvent.org). Report the path of this folder below.

**!!!** Be careful that the path is with "/" instead of "\\"

In [3]:
path_datasets = "C:/Users/jeromea/Downloads/ecoinvent_3.9.1_cutoff_ecoSpold02/datasets"

The name of the imported database will be:

In [4]:
ei_db_name = "ecoinvent_391_cutoff"

After checking that the same database is not imported twice, the import is done is several steps:

1. First, all the datasets are loaded with `bi.SingleOutputEcospold2Importer`,
2. Then, the data are changed based on several "strategies" which are functions to modify the data,
3. You can check that things went well with `statistics()`, especially that there is "0 unlinked exchanges"
4. Finally, the database, for now only in the memory for your python session, is saved to your computer with `write_database()`

In [5]:
if ei_db_name in bd.databases:
    print("Database has already been imported.")
else:
    ei_importer = bi.SingleOutputEcospold2Importer(path_datasets, ei_db_name)
    ei_importer.apply_strategies()
    ei_importer.statistics() 
    ei_importer.write_database()

Database has already been imported.


You can see all the metadata (i.e., the properties for describing the database) of the imported database with:

Save the ecoinvent database as `ei_db`:

## Accessing activities and flows

To search for activities in a database:

To return (or "get") a specific activity object:

To look at all information stored in the activity:

To look at the exchanges:

# Manual creation of a database

## Create the database

Function to delete a database if it exists in the current project:

In [6]:
def reset_brightway_database(db_name):
    if db_name in bd.databases:
        del bd.databases[db_name]

Create a new database:

Let's check that it was created:

## Add activities

Function to delete an activity:

In [7]:
def reset_activity(act_name, database=motor_db):
    for activity in [act for act in motor_db if act['name']==act_name]:
        activity.delete()

NameError: name 'motor_db' is not defined

Create an activity:

Create an exchange:

**Inputs from ecoinvent**

First, let get the necessary activities from ecoinvent:

In [29]:
act_steel_low_alloyed = ei_db.get(name="market for steel, low-alloyed", location="GLO")

In [30]:
act_copper = ei_db.get(name="market for copper, cathode", location="GLO")

In [31]:
act_insulation = ei_db.get(name="market for epoxy resin, liquid", location="RER")

In [32]:
act_elec = ei_db.get(name="market for electricity, high voltage", location="SE")

In [33]:
act_incineration_insulation = ei_db.get(name="treatment of waste plastic, mixture, municipal incineration with fly ash extraction", location="CH", unit="kilogram")

In [34]:
act_incineration_copper = ei_db.get(name="treatment of copper in car shredder residue, municipal incineration with fly ash extraction", location="CH")

**Parameters**

Then the value of different parameters useful to create the activities.

In [35]:
qt_steel_unalloyed = 26600 #kg
qt_steel_low_alloyed = 4100 #kg
qt_copper = 3600 #kg
qt_insulation = 400 #kg
weight_motor = qt_steel_unalloyed + qt_steel_low_alloyed + qt_copper + qt_insulation
output_power = 16000 #kW
op_hours = 8400 #hours per year
lifetime = 20 #years
efficiency = 0.973

**Activities creation**

# First LCA calculation

## Selection of impact assessment methods

List of all available methods:

For searching in the list of methods, use conditions in the list:

Methods can be selected when knowing the tuple for their name:

Or by selecting the items in a list:

## Impact assessment

Calculate another impact category:

Here is a function to calculate the impact for several impact categories and return the results in a table:

In [8]:
def lca_results(fu, list_methods):
    list_units = [bd.methods.get(method)["unit"] for method in list_methods]
    lca = bc.LCA(demand=fu, method=list_methods[0])
    lca.lci()
    lca.lcia()
    list_scores = [lca.score]
    for method in list_methods[1:]:
        lca.switch_method(method)
        lca.lcia()
        list_scores.append(lca.score)
    return pd.DataFrame({"score": list_scores, "unit": list_units, "method": list_methods})

## Basic contribution analysis

Brightway provides functions for contribution analysis with the top processes:

And with the top contributing elementary flows:

## Customised contribution analysis

Here I provide functions for contribution analysis for the processes in the created database:

In [9]:
def table_recursive_multi_calculation_dbparent_cut(activity, lcia_methods, amount=1, level=0, max_level=3, db_parent=None, db_cut=['ecoinvent'], act_parent={"name": None}):
    """
    Returns a table (DataFrame) with the contribution tree for a given activity and various LCIA methods.
    The contribution tree does not contain sub-activities if the activity is from a database in db_cut (to avoid 
    having details on ecoinvent datasets and to stay in my own model) and has the maximum level max_level.
    Columns for the returned table: level, activity, activity parent, database parent, and str(method) for all LCIA method explored.
    """
    
    lca_score = lca_results({activity: amount}, lcia_methods)
    table = pd.DataFrame([[level, activity["name"], act_parent["name"], db_parent]+ list(lca_score["score"])], columns=["level", "activity", "activity parent", "database parent"]+[str(m) for m in lcia_methods])
    if level < max_level:
        db_parent = activity['database']
        go_forward = True
        for stop_word in db_cut:
            if stop_word in db_parent:
                go_forward = False
        if go_forward:
            for exc in activity.technosphere():
                table = pd.concat([table, table_recursive_multi_calculation_dbparent_cut(
                    activity=exc.input, 
                    lcia_methods=lcia_methods, 
                    amount=amount * exc['amount'], 
                    level=level + 1, 
                    max_level=max_level,
                    db_parent = db_parent,
                    act_parent = activity
                )], ignore_index=True)
    return table

In [10]:
def table_recursive_multi_calculation_dbcut(activity, lcia_methods, amount=1, level=0, max_level=3, db_activity='motor_case', db_cut=['ecoinvent', 'biosphere']):
    """
    Returns a table (DataFrame) with the contribution tree for a given activity and various LCIA methods.
    The contribution tree does not contain activities if they are from a database in db_cut (to avoid 
    having details on ecoinvent datasets and to stay in my own model) and has the maximum level max_level.
    Columns for the returned table: level, activity, database, and str(method) for all LCIA method explored.
    """
    
    lca_score = lca_results({activity: amount}, lcia_methods)
    table = pd.DataFrame([[level, activity["name"], db_activity]+ list(lca_score["score"])], columns=["level", "activity", "database"]+[str(m) for m in lcia_methods])
    if level < max_level:
        for exc in activity.technosphere():
            db_exc = exc.input['database']
            go_forward = True
            for stop_word in db_cut:
                if stop_word in db_exc:
                    go_forward = False
            if go_forward:
                table = pd.concat([table, table_recursive_multi_calculation_dbcut(
                    activity=exc.input, 
                    lcia_methods=lcia_methods, 
                    amount=amount * exc['amount'], 
                    level=level + 1, 
                    max_level=max_level,
                    db_activity = db_exc
                )], ignore_index=True)
    return table

## Figures

Here is a way to generate a box plot figure.

In [11]:
own_colors = {
    "motor production":"#1f78b4", #blue
    "motor use":"#969696", #gray
    "motor EoL treatment": "#41ae76", #green
    "motor testing":"#fff7bc" #yellow
}

In [65]:
table_results = 

In [12]:
def basic_bar_chart(table_results, legend_x_axis, own_colors=own_colors):
    table_figure = table_results.set_index(table_results["activity"])
    number_methods = table_figure.shape[1] - 3

    fig, ax = plt.subplots(1, number_methods)
    x_axis = [0]

    for n_fig, method in enumerate(table_figure.columns[3:]):
        for activity in table_figure.index:
            if n_fig == 0:
                ax[n_fig].bar(
                    x_axis, 
                    table_figure.loc[activity, method], 
                    label=activity, 
                    color=own_colors[activity]
                )
            else:
                ax[n_fig].bar(
                    x_axis, 
                    table_figure.loc[activity, method], 
                    color=own_colors[activity]
                )
    
    fig.tight_layout()
    for n_fig in range(number_methods):
        ax[n_fig].grid(True, axis='y', linewidth=0.4)
        ax[n_fig].set_xticks(x_axis)
        ax[n_fig].set_xticklabels([legend_x_axis[n_fig]], fontsize=14)
    
    fig.legend(loc='lower center', ncol=5)
    plt.subplots_adjust(bottom=0.15)
    
    return plt.show()

# Different scenarios

## Data

In [81]:
data_low_efficiency = {
    "qt_steel_unalloyed": 26600, #kg
    "qt_steel_low_alloyed": 4100, #kg
    "qt_copper": 3600, #kg
    "qt_insulation": 400, #kg
    "weight_motor": 26600+4100+3600+400, #kg
    "lifetime": 20, #years
    "efficiency": 0.973
}

In [82]:
data_high_efficiency = {
    "qt_steel_unalloyed": 18000, #kg
    "qt_steel_low_alloyed": 9200, #kg
    "qt_copper": 3800, #kg
    "qt_insulation": 300, #kg
    "weight_motor": 18000+9200+3800+300, #kg
    "lifetime": 20, #years
    "efficiency": 0.9835
}

In [83]:
data_repair_low_efficiency = {
    "qt_steel_unalloyed": 26600, #kg
    "qt_steel_low_alloyed": 4100, #kg
    "qt_copper": 3600*1.5, #kg
    "qt_insulation": 400*1.5, #kg
    "weight_motor": 26600+4100+3600*1.5+400*1.5, #kg
    "lifetime": 40, #years
    "efficiency": 0.973
}

In [84]:
data_repair_high_efficiency = {
    "qt_steel_unalloyed": 18000, #kg
    "qt_steel_low_alloyed": 9200, #kg
    "qt_copper": 3800*1.5, #kg
    "qt_insulation": 300*1.5, #kg
    "weight_motor": 18000+9200+3800*1.5+300*1.5, #kg
    "lifetime": 40, #years
    "efficiency": 0.9835
}

## Generation of results

## Figures

In [156]:
def figure_comparison_scenarios(table_results_scenarios, legend_x_axis):
    number_methods = 2

    fig, ax = plt.subplots(1, number_methods)
    x_axis = [i for i in range(len(dict_scenarios))]

    for index in table_results_scenarios.index:
        ax[index%2].bar(x_axis[index//2], table_results_scenarios.loc[index, "score"])
    
    fig.tight_layout()
    for n_fig in range(number_methods):
        ax[n_fig].grid(True, axis='y', linewidth=0.4)
        ax[n_fig].set_xticks(x_axis)
        ax[n_fig].set_xticklabels(legend_x_axis, rotation=90)
        ax[n_fig].set_ylabel(table_results_scenarios.loc[n_fig, "unit"])

    plt.subplots_adjust(wspace=0.3)
    
    return plt.show()